tensorflow 2.x 전문가용 DNN model 구축
  - tensorflow2.0 저수준 API
  - Dataset class 이용 : 공급 data 생성
  - 순방향 step : 회귀방정식 연산 -> 예측치 -> loss
  - 역방향 step : 자동미분계산 -> w, b update(model 최적화)
  - 손실함수, 최적화, 모델평가 API

In [1]:
import tensorflow as tf # ver2.0
from tensorflow.python.data import Dataset # dataset 생성
from tensorflow.keras.layers import Dense, Flatten # layer 추가
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import datasets # mnist, cifar10

In [2]:
# 1. dataset load
mnist = datasets.mnist
(x_train,y_train),(x_val,y_val) = mnist.load_data()

x_train.shape
x_val.shape

'''
# image 2d -> 1d
x_train = x_train.reshape(-1, 28*28)
x_val = x_val.reshape(-1, 784)
'''

# image 정규화 -> 실수형
x_train = x_train/225.
x_val = x_val/225.

# labels
y_train # [5, 0, 4, ..., 5, 6, 8] : integer




# 2. Dataset 생성
train_ds = Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
train_ds
test_ds = Dataset.from_tensor_slices((x_val, y_val)).shuffle(10000).batch(32)
test_ds




inputshape = (28,28)

# 3. 순방향 step : 연산 -> (예측치 vs 관측치) -> loss
# model class
class Model(tf.keras.Model) :   # 자식 클ㄹ스(부모클래스)   
    def __init__(self) :  # 생성자
        super().__init__()  # 부모 생성자 호출
        # x, b 자동 초기화. 따로 설정해주지 않아도 됨
        # DNN layer
        # 2d -> 1d
        self.d0 = Flatten(input_shape=inputshape) # flatten layer
        self.d1 = Dense(128, activation='relu')  # hidden layer 1
        self.d2 = Dense(64, activation='relu')  # hidden layer2
        self.d3 = Dense(10, activation='softmax')  # output layer
    
    def call(self, inputs) :  # 메서드 재정의
        # 회귀방정식은 생략(자동으로 만들어짐)
        x = self.d0(inputs)
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)  # 예측치(확률) 반환



# 4. loss functin : 손실 함수 : 오차 반환
loss = losses.SparseCategoricalCrossentropy(from_logits=True)

# y_true (integer) vs y_pred(prop) : from_logits=True


import numpy as np

# 손실이 작은 경우
y_true = np.array([0,2]) # 정답 : 10진수
y_pred = np.array([[0.9,0.02,0.08],[0.1,0.1,0.8]]) # 예측치 : 확률

loss(y_true, y_pred).numpy() # 손실함수 : 0.6538635492324829
  
# 손실이 큰 경우
y_true = np.array([0,1]) # 정답 : 10진수
y_pred = np.array([[0.9,0.02,0.08],[0.1,0.1,0.8]]) # 예측치 : 확률

loss(y_true, y_pred).numpy() # 손실함수 : 1.0038635730743408
  


# 5. model & optimizer
model = Model()
optimizer = optimizers.Adam()




# 6. model 평가 : loss, accuracy -> 1 epoch 단위로 평가
train_loss = metrics.Mean()  # loss mean
train_acc = metrics.SparseCategoricalAccuracy()  # accuracy

val_loss = metrics.Mean()  # loss mean
val_acc = metrics.SparseCategoricalAccuracy()  # accuracy



In [3]:


# 7. 역방향 step : 자동 미분계수
@tf.function # 연산속도 향상
def train_step(images, labels) :
    with tf.GradientTape() as tape:
        # 1) 순방향 : loss 계산
        preds = model(images) # model.call(images) : 예측치
        loss_value = loss(labels, preds) # 손실함수 (y_true, y_pred)
        
        # 2) 역방향 : 손실값 -> [Model.W, Model.B]
        grad = tape.gradient(loss_value, model.trainable_variables)
        # 기울기 -> 최적화 객체 반영
        optimizer.apply_gradients(zip(grad,model.trainable_variables))
        
        # 3) 1epoch -> loss, accuracy save
        train_loss(loss_value) # loss mean
        train_acc(labels, preds) # accuracy
    

@tf.function # 연산속도 향상
def test_step(images, labels) :
    with tf.GradientTape() as tape:
        # 1) 순방향 : loss 계산
        preds = model(images) # model.call(images) : 예측치
        loss_value = loss(labels, preds) # 손실함수 (y_true, y_pred)
        
        # 2) 역방향 : 없음
        
        # 3) 1epoch -> loss, accuracy save
        val_loss(loss_value) # loss mean
        val_acc(labels, preds) # accuracy




# 8. model training
epochs = 10

for epoch in range(epochs): 
    
    # model train
    for images, labels in train_ds:
        train_step(images, labels)
        
    # model val
    for images, labels in test_ds:
        test_step(images, labels)
        
    form = "epoch = {}, train loss = {:.6f}, train acc = {:.6f}, val loss = {:.6f}, val acc = {:.6f}"
    print(form.format(epoch+1, train_loss.result(),
                      train_acc.result(),
                      val_loss.result(),
                      val_acc.result()))
        


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch = 1, train loss = 1.560654, train acc = 0.911300, val loss = 1.513195, val acc = 0.950300
epoch = 2, train loss = 1.535758, train acc = 0.932133, val loss = 1.511368, val acc = 0.951950
epoch = 3, train loss = 1.524015, train acc = 0.942089, val loss = 1.506330, val acc = 0.956533
epoch = 4, train loss = 1.516525, train acc = 0.948646, val loss = 1.504486, val acc = 0.958300
epoch = 5, train loss = 1.511065, train acc = 0.953430, val loss = 1.501717, val acc = 0.960860
epoch = 6, train loss = 1.50